<a href="https://colab.research.google.com/github/jpvsalomao/Cryptocrimes/blob/main/Cryptocrimes_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Run below commands
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz
!tar xf spark-3.1.2-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [ ]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
from pyspark import SparkConf, SparkContext
from datetime import datetime, date, timedelta
from dateutil import relativedelta
from pyspark.sql import SQLContext, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import split
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from pyspark.sql.functions import to_timestamp, to_date
from pyspark.sql import functions as F  
from pyspark.sql.functions import collect_list, collect_set, concat, first, array_distinct, col, size, expr
from pyspark.sql import DataFrame 
import random
import pandas as pd
from pyspark.sql.functions import max
from pyspark.sql.functions import desc

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
spark = SparkSession.builder.appName('eda').getOrCreate()

In [ ]:
file1 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0x12d66f87a04a9e220743712ce6d9bb1b5616b8fc.csv")
 
file2 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0x3a1d526d09b7e59fd88de4726f68a8246ddc2742.csv")


file3 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0x722122df12d4e14e13ac3b6895a86e84145b6967.csv")

file4 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0x910cbd523d972eb0a6f4cae4618ad62622b39dbf.csv")

file5 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0xA160cdAB225685dA1d56aa342Ad8841c3b53f291.csv")

file6 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0xd90e2f925da726b50c4ed8d0fb90ad053324f31b.csv")

file7 = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "false")\
    .csv("/content/drive/MyDrive/Crypto launders/token_0x47ce0c6ed5b0ce3d3a51fdb1c52dc66a7c3c2936.csv")

tokens_to_scrape = spark\
    .read\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .csv("/content/drive/MyDrive/Crypto launders/tokenListToScrape.csv")


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount = True)

Mounted at /content/drive


# Nova seção

In [ ]:
#### combining all the files ####

df_concat = file1.union(file2)
df_concat = df_concat.union(file3)
df_concat = df_concat.union(file4)
df_concat = df_concat.union(file5)
df_concat = df_concat.union(file6)
combined_transactions = df_concat.union(file7)

In [ ]:
columns = ['TxnHash', 'method', 'block', 'datetime', 'from', 'to', 'value', 'TxnFees']

combined_transactions = combined_transactions.withColumnRenamed('_c0', 'TxnHash')\
                      .withColumnRenamed('_c1', 'method')\
                      .withColumnRenamed('_c2', 'block')\
                      .withColumnRenamed('_c3', 'datetime')\
                      .withColumnRenamed('_c4', 'from')\
                      .withColumnRenamed('_c5', 'to')\
                      .withColumnRenamed('_c6', 'value')\
                      .withColumnRenamed('_c7', 'TxnFees')

In [ ]:
### removing the word ether from the column ###

combined_transactions = combined_transactions.withColumn("Value", split(col("value"), " Ether").getItem(0))

In [ ]:
combined_transactions = combined_transactions.withColumn("date", split(col("datetime"), " ").getItem(0))\
                                              .withColumn("time", split(col("datetime"), " ").getItem(1))

In [ ]:
combined_transactions.printSchema()

root
 |-- TxnHash: string (nullable = true)
 |-- method: string (nullable = true)
 |-- block: integer (nullable = true)
 |-- datetime: string (nullable = true)
 |-- from: string (nullable = true)
 |-- to: string (nullable = true)
 |-- Value: string (nullable = true)
 |-- TxnFees: double (nullable = true)
 |-- date: string (nullable = true)
 |-- time: string (nullable = true)



In [ ]:
#### changing the data type of the combined transaction file ####

combined_transactions = combined_transactions.withColumn("value", 
                                  combined_transactions["Value"]
                                  .cast('float'))

In [ ]:
combined_transactions.show()

+--------------------+----------+--------+--------------+--------------------+--------------------+-----+----------+--------+-----+
|             TxnHash|    method|   block|      datetime|                from|                  to|value|   TxnFees|    date| time|
+--------------------+----------+--------+--------------+--------------------+--------------------+-----+----------+--------+-----+
|0x5019ed6a4024ecb...|   Deposit|14564890|11/04/22 08:11|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.05442878|11/04/22|08:11|
|0x271eb419fffbee0...|   Deposit|14563771|11/04/22 03:11|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.04038231|11/04/22|03:11|
|0xbcc67ef16dab428...|  Withdraw|14507337|02/04/22 08:11|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01665735|02/04/22|08:11|
|0x9415ee0a3ce8088...|  Withdraw|14452802|24/03/22 20:11|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01302876|24/03/22|20:11|
|0x6abfc8bbdff17ae...|  Withdraw|14452675|24/03/22 19:11|0xe0f3772995bbac1..

In [ ]:
#### converting it into sql table ####

combined_transactions.createOrReplaceTempView("combined_sql")

In [ ]:
sql_code = spark.sql("""
  SELECT 
      *
  FROM combined_sql
  LIMIT 10
  """)

sql_code.show(10)

+--------------------+--------+--------+--------------+--------------------+--------------------+-----+----------+--------+-----+
|             TxnHash|  method|   block|      datetime|                from|                  to|value|   TxnFees|    date| time|
+--------------------+--------+--------+--------------+--------------------+--------------------+-----+----------+--------+-----+
|0x5019ed6a4024ecb...| Deposit|14564890|11/04/22 08:11|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.05442878|11/04/22|08:11|
|0x271eb419fffbee0...| Deposit|14563771|11/04/22 03:11|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.04038231|11/04/22|03:11|
|0xbcc67ef16dab428...|Withdraw|14507337|02/04/22 08:11|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01665735|02/04/22|08:11|
|0x9415ee0a3ce8088...|Withdraw|14452802|24/03/22 20:11|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01302876|24/03/22|20:11|
|0x6abfc8bbdff17ae...|Withdraw|14452675|24/03/22 19:11|0xe0f3772995bbac1...|Tornado.Cash: 

In [ ]:
sql_code = spark.sql("""
  SELECT 
      TxnHash,
      method,
      block,
      from,
      to, 
      value,
      TxnFees,
      time,
      to_date(date, 'dd/MM/yy') AS date
  FROM combined_sql
  """)

sql_code.show(10)

+--------------------+--------+--------+--------------------+--------------------+-----+----------+-----+----------+
|             TxnHash|  method|   block|                from|                  to|value|   TxnFees| time|      date|
+--------------------+--------+--------+--------------------+--------------------+-----+----------+-----+----------+
|0x5019ed6a4024ecb...| Deposit|14564890|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.05442878|08:11|2022-04-11|
|0x271eb419fffbee0...| Deposit|14563771|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.04038231|03:11|2022-04-11|
|0xbcc67ef16dab428...|Withdraw|14507337|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01665735|08:11|2022-04-02|
|0x9415ee0a3ce8088...|Withdraw|14452802|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01302876|20:11|2022-03-24|
|0x6abfc8bbdff17ae...|Withdraw|14452675|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01943416|19:11|2022-03-24|
|0xe67845a9ee23eb7...|Withdraw|14449957|0xe0f3772995bbac1...|Tor

In [ ]:
sql_code.createOrReplaceTempView("combined_sql2")

In [ ]:
sql_code = spark.sql("""
  SELECT 
      *
  FROM combined_sql2
  """)

sql_code.show()

+--------------------+----------+--------+--------------------+--------------------+-----+----------+-----+----------+
|             TxnHash|    method|   block|                from|                  to|value|   TxnFees| time|      date|
+--------------------+----------+--------+--------------------+--------------------+-----+----------+-----+----------+
|0x5019ed6a4024ecb...|   Deposit|14564890|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.05442878|08:11|2022-04-11|
|0x271eb419fffbee0...|   Deposit|14563771|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.04038231|03:11|2022-04-11|
|0xbcc67ef16dab428...|  Withdraw|14507337|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01665735|08:11|2022-04-02|
|0x9415ee0a3ce8088...|  Withdraw|14452802|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01302876|20:11|2022-03-24|
|0x6abfc8bbdff17ae...|  Withdraw|14452675|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01943416|19:11|2022-03-24|
|0xe67845a9ee23eb7...|  Withdraw|14449957|0xe0f3

In [ ]:
sql_code = spark.sql("""
  SELECT 
      COUNT(*)
  FROM combined_sql2
  """)

sql_code.show()

+--------+
|count(1)|
+--------+
|  169621|
+--------+



In [ ]:
### value of transaction ###

sql_code = spark.sql("""
  SELECT 
      from,
      SUM(value) AS total_value
  FROM combined_sql2
  GROUP BY 1
  ORDER BY 2 DESC
  """)

sql_code.show(10)

+--------------------+-----------+
|                from|total_value|
+--------------------+-----------+
|    Bitmart Hacker 2|    21170.0|
|0x5481eccfce4303e...|    15803.0|
|0x7a7ebe060943650...|    12720.0|
|0x34a17418cec67b8...|    11500.0|
|0x1b7baa734c00298...|    10800.0|
|0xe0e484dfa7f3aa3...|    10200.0|
|            sifu.eth|    10061.0|
|0xd88189f7dee6e5d...|    10000.0|
|0x20d9e73aafe69a3...|     9960.0|
|Vee Finance Explo...|     8801.0|
+--------------------+-----------+
only showing top 10 rows



In [ ]:
### total number of transaction ###

sql_code = spark.sql("""
  SELECT 
      from,
      COUNT(value) AS total_transactions
  FROM combined_sql2
  GROUP BY 1
  ORDER BY 2 DESC
  """)

sql_code.show(10)

+--------------------+------------------+
|                from|total_transactions|
+--------------------+------------------+
|0xd6187b4a0f51355...|              6885|
|0xbe4d1e137a24af0...|              6827|
|0x20bb3095a4852f4...|              6045|
|0xa0f0287683e820f...|              5860|
|0x49136693081f2c1...|              5758|
|0x7d3bb46c78b0c49...|              3919|
|0x03ebd2ea2b9f236...|              2914|
|0x2d50dbcc960bec8...|              2522|
|      poarelayer.eth|              1846|
|0xa8af5873c59dca7...|              1585|
+--------------------+------------------+
only showing top 10 rows



In [ ]:
#### checking if there is an interesection between top 1000 user ids in both from and to in terms of total value #####

intersection_sql = spark.sql("""
WITH cte1 AS (
  SELECT 
      from,
      SUM(value) AS tv1
  FROM combined_sql2
  GROUP BY 1
  ORDER BY 2 DESC
),
cte2 AS (
  SELECT 
      to,
      SUM(value) AS tv2
  FROM combined_sql2
  GROUP BY 1
  ORDER BY 2 DESC
)
SELECT
    cte1.from,
    cte2.to,
    cte1.tv1 AS value_sent,
    cte2.tv2 AS value_received
FROM cte1
INNER JOIN cte2
ON cte1.from = cte2.to
""")
intersection_sql.show()

+--------------+--------------+------------------+-----------------+
|          from|            to|        value_sent|   value_received|
+--------------+--------------+------------------+-----------------+
|em3tornado.eth|em3tornado.eth|0.2818463023286313|1.493749976158142|
+--------------+--------------+------------------+-----------------+



No one in 'from' is present inside 'to' and which is how it should be since this only contains transaction done to tornado cash i.e. deposit and not withdrawal. 

In [ ]:
#### year wise value ####

year_sql = spark.sql("""
  SELECT 
      year(date) AS year,
      month(date) AS month,
      SUM(value) AS total_value
  FROM combined_sql2
  GROUP BY 1, 2
  ORDER BY 1, 2
  """)

year_sql.show(100)

+----+-----+--------------------+
|year|month|         total_value|
+----+-----+--------------------+
|2019|   12|   2568.200000755489|
|2020|    1|   9524.800000861287|
|2020|    2|   8516.300000600517|
|2020|    3|   5361.201000554371|
|2020|    4|  13060.800000622869|
|2020|    5|  25428.200000479817|
|2020|    6|  14535.400000408292|
|2020|    7|  25590.200100360606|
|2020|    8|   34913.80000023544|
|2020|    9|   54464.80000026524|
|2020|   10|  148754.20000119507|
|2020|   11|  106461.40010119806|
|2020|   12|  56437.950000640005|
|2021|    1|  200.10000000149012|
|2021|    2|   302.1007769564749|
|2021|    3|6.637100013904274E-4|
|2021|    4|  100.10000000149012|
|2021|    5|  120.10000000149012|
|2021|    6|  12625.208360181306|
|2021|    7|   111612.7006117191|
|2021|    8|  153499.33085846156|
|2021|    9|  119479.53479407169|
|2021|   10|  107458.19046464656|
|2021|   11|   97771.43301149149|
|2021|   12|  139573.06948774913|
|2022|    1|  113939.87440695427|
|2022|    2|  

The activity in the last 6-7 months has exploded. 

In [ ]:
sql_code = spark.sql("""
  SELECT 
      DISTINCT TxnHash 
  FROM combined_sql2
  WHERE to LIKE "Tornado.Cash:%ETH"
  AND method = "Withdraw"
  """)

sql_code.show(20)

+--------------------+
|             TxnHash|
+--------------------+
|0x3766535adf78cd2...|
|0xea7c000c8a6a3cf...|
|0xb14524ec40ec59e...|
|0xb90c6c946fb7f39...|
|0xcee918b9d619d39...|
|0xbfc8ac90916cd7d...|
|0x54fe8c964bd5355...|
|0xe40eedfd646c03d...|
|0x7dcec300d85d72e...|
|0x7555116847ae2eb...|
|0xb74b874371b1771...|
|0xf0b752297ecc469...|
|0xcb9d19da44fce40...|
|0xe5bb17e135adbd5...|
|0x16d02d2d835d9d4...|
|0x9fb0f33af635cac...|
|0xe7d9a56b208cd06...|
|0xc6629cd0c3c4d3a...|
|0xbf8ebf8b5b5b2ed...|
|0xa7077d0b3e88596...|
+--------------------+
only showing top 20 rows



In [ ]:
year_sql = spark.sql("""
  SELECT 
      year(date) AS year,
      month(date) AS month,
      SUM(value) AS total_value,
      COUNT(DISTINCT TxnHash) AS transactions,
      COUNT(DISTINCT from) AS users,
      ROUND(COUNT(DISTINCT TxnHash) / COUNT(DISTINCT from), 2) AS avg_transactions,
      ROUND(SUM(value), 2) AS total_value
  FROM combined_sql2
  WHERE year(date) >= 2021
  GROUP BY 1, 2
  ORDER BY 1, 2
  """)

year_sql.show(100)

+----+-----+--------------------+------------+-----+----------------+-----------+
|year|month|         total_value|transactions|users|avg_transactions|total_value|
+----+-----+--------------------+------------+-----+----------------+-----------+
|2021|    1|  200.10000000149012|           5|    3|            1.67|      200.1|
|2021|    2|   302.1007769564749|           7|    5|             1.4|      302.1|
|2021|    3|6.637100013904274E-4|           1|    1|             1.0|        0.0|
|2021|    4|  100.10000000149012|           2|    2|             1.0|      100.1|
|2021|    5|  120.10000000149012|           6|    4|             1.5|      120.1|
|2021|    6|  12625.208360181306|        1541|  323|            4.77|   12625.21|
|2021|    7|   111612.7006117191|       15485| 2326|            6.66|   111612.7|
|2021|    8|  153499.33085846156|       13953| 2448|             5.7|  153499.33|
|2021|    9|  119479.53479407169|       12308| 2166|            5.68|  119479.53|
|2021|   10|  10

In [ ]:
year_sql = spark.sql("""
  SELECT 
      year(date) AS year,
      month(date) AS month,
      date,
      SUM(value) AS total_value,
      COUNT(DISTINCT TxnHash) AS transactions,
      COUNT(DISTINCT from) AS users,
      ROUND(COUNT(DISTINCT TxnHash) / COUNT(DISTINCT from), 2) AS avg_transactions,
      ROUND(SUM(value), 2) AS total_value
  FROM combined_sql2
  WHERE year(date) = 2021
        AND month(date) BETWEEN 4 and 7 
  GROUP BY 1, 2, 3
  ORDER BY 3
  """)

year_sql.show()

+----+-----+----------+-------------------+------------+-----+----------------+-----------+
|year|month|      date|        total_value|transactions|users|avg_transactions|total_value|
+----+-----+----------+-------------------+------------+-----+----------------+-----------+
|2021|    4|2021-04-24|0.10000000149011612|           1|    1|             1.0|        0.1|
|2021|    4|2021-04-30|              100.0|           1|    1|             1.0|      100.0|
|2021|    5|2021-05-06|0.10000000149011612|           2|    2|             1.0|        0.1|
|2021|    5|2021-05-07|                0.0|           1|    1|             1.0|        0.0|
|2021|    5|2021-05-11|              120.0|           3|    1|             3.0|      120.0|
|2021|    6|2021-06-05|0.10000000149011612|           2|    2|             1.0|        0.1|
|2021|    6|2021-06-12|0.10000000149011612|           1|    1|             1.0|        0.1|
|2021|    6|2021-06-26|0.10000000149011612|           1|    1|             1.0| 

I went through their twitter account and this increase is actually preceeded by listing on BNB and Polygon blockchain networks and that's when the trasactions expanded. 

In [ ]:
# from google.colab import files

# df = sql_code.toPandas()
# df.to_csv('txnHash_withdraw.csv', encoding = 'utf-8-sig') 
# files.download('txnHash_withdraw.csv')

In [ ]:
year_sql = spark.sql("""
  SELECT 
      year(date) AS year,
      month(date) AS month,
      SUM(value) AS total_value,
      COUNT(DISTINCT TxnHash) AS transactions,
      COUNT(DISTINCT from) AS users,
      ROUND(COUNT(DISTINCT TxnHash) / COUNT(DISTINCT from), 2) AS avg_transactions,
      ROUND(SUM(value), 2) AS total_value
  FROM combined_sql2 
  GROUP BY 1, 2
  ORDER BY 1,2
  """)

year_sql.show()

+----+-----+--------------------+------------+-----+----------------+-----------+
|year|month|         total_value|transactions|users|avg_transactions|total_value|
+----+-----+--------------------+------------+-----+----------------+-----------+
|2019|   12|   2568.200000755489|         923|  272|            3.39|     2568.2|
|2020|    1|   9524.800000861287|        2099|  586|            3.58|     9524.8|
|2020|    2|   8516.300000600517|        1898|  485|            3.91|     8516.3|
|2020|    3|   5361.201000554371|        1646|  410|            4.01|     5361.2|
|2020|    4|  13060.800000622869|        2245|  439|            5.11|    13060.8|
|2020|    5|  25428.200000479817|        2753|  489|            5.63|    25428.2|
|2020|    6|  14535.400000408292|        2292|  561|            4.09|    14535.4|
|2020|    7|  25590.200100360606|        2474|  594|            4.16|    25590.2|
|2020|    8|   34913.80000023544|        2656|  611|            4.35|    34913.8|
|2020|    9|   5

In [ ]:
sql_code = spark.sql("""
  SELECT 
      *
  FROM combined_sql2
  """)

sql_code.show(5)

+--------------------+--------+--------+--------------------+--------------------+-----+----------+-----+----------+
|             TxnHash|  method|   block|                from|                  to|value|   TxnFees| time|      date|
+--------------------+--------+--------+--------------------+--------------------+-----+----------+-----+----------+
|0x5019ed6a4024ecb...| Deposit|14564890|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.05442878|08:11|2022-04-11|
|0x271eb419fffbee0...| Deposit|14563771|         coinbro.eth|Tornado.Cash: 0.1...|  0.1|0.04038231|03:11|2022-04-11|
|0xbcc67ef16dab428...|Withdraw|14507337|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01665735|08:11|2022-04-02|
|0x9415ee0a3ce8088...|Withdraw|14452802|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01302876|20:11|2022-03-24|
|0x6abfc8bbdff17ae...|Withdraw|14452675|0xe0f3772995bbac1...|Tornado.Cash: 0.1...|  0.0|0.01943416|19:11|2022-03-24|
+--------------------+--------+--------+--------------------+---

In [ ]:
sql_code = spark.sql("""
  SELECT 
      to,
      method,
      count(value)
  FROM combined_sql2
  where method in ('Withdraw','Deposit')
  group by 1,2
  order by to
  """)

sql_code.show()

+--------------------+--------+------------+
|                  to|  method|count(value)|
+--------------------+--------+------------+
|Tornado.Cash: 0.1...|Withdraw|        4234|
|Tornado.Cash: 0.1...| Deposit|        5254|
| Tornado.Cash: 1 ETH|Withdraw|        5967|
| Tornado.Cash: 1 ETH| Deposit|        6397|
|Tornado.Cash: 10 ETH| Deposit|        6979|
|Tornado.Cash: 10 ETH|Withdraw|        6729|
|Tornado.Cash: 100...| Deposit|        4358|
|Tornado.Cash: 100...|Withdraw|        3999|
| Tornado.Cash: Proxy|Withdraw|       52261|
| Tornado.Cash: Proxy| Deposit|       46682|
|Tornado.Cash: Router| Deposit|       13152|
|Tornado.Cash: Router|Withdraw|       13422|
+--------------------+--------+------------+



In [ ]:
df = spark.read.format("csv") \
                    .option("header", "true") \
                    .option("inferSchema","true") \
                    .load('/content/drive/MyDrive/first_degree_data')